In [1]:
import napari
import numpy as np
from pathlib import Path
from tifffile import imread

from napari_cellseg3d.dev_scripts import evaluate_labels as eval
from napari_cellseg3d.utils import resize
from napari_cellseg3d.code_models.model_instance_seg import (
    binary_connected,
    binary_watershed,
    voronoi_otsu,
    to_semantic,
)

In [2]:
viewer = napari.Viewer()

In [3]:
im_path = Path("C:/Users/Cyril/Desktop/test/instance_test")
# prediction_path = str(im_path / "trailmap_ms/trailmap_pred.tif")
prediction_path = str(im_path / "pred.tif")
gt_labels_path = str(im_path / "labels_relabel_unique.tif")

prediction = imread(prediction_path)
gt_labels = imread(gt_labels_path)

zoom = (1 / 5, 1, 1)
prediction_resized = resize(prediction, zoom)
# prediction_resized = prediction # for trailmap
gt_labels_resized = resize(gt_labels, zoom)


viewer.add_image(prediction_resized, name="pred", colormap="inferno")
viewer.add_labels(gt_labels_resized, name="gt")

<Labels layer 'gt' at 0x201a14b1a00>

In [4]:
from napari_cellseg3d.utils import dice_coeff

semantic_gt = to_semantic(gt_labels_resized.copy())
semantic_pred = to_semantic(prediction_resized.copy())

viewer.add_image(semantic_gt, colormap='bop blue')
viewer.add_image(semantic_pred, colormap='red')

dice_coeff(
    semantic_gt,
    prediction_resized
)

0.8592223181276479

In [5]:
from napari_cellseg3d.dev_scripts.correct_labels import relabel

# gt_corrected = relabel(prediction_path, gt_labels_path, go_fast=False)

In [6]:
# eval.evaluate_model_performance(gt_labels_resized, gt_labels_resized)

In [7]:
print(prediction_resized.shape)
print(gt_labels_resized.shape)
print(np.unique(gt_labels_resized).shape[0])

(25, 64, 64)
(25, 64, 64)
125


In [8]:
connected = binary_connected(prediction_resized, thres_small=2)
viewer.add_labels(connected, name="connected")

<Labels layer 'connected' at 0x201acaf1250>

In [9]:
eval.evaluate_model_performance(gt_labels_resized, connected)

2023-03-31 15:37:19,775 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3699.66it/s]

2023-03-31 15:37:19,812 - Calculating the number of neurons not found...
2023-03-31 15:37:19,815 - Percent of non-fused neurons found: 52.00%
2023-03-31 15:37:19,816 - Percent of fused neurons found: 36.80%
2023-03-31 15:37:19,817 - Overall percent of neurons found: 88.80%


(65,
 46,
 13,
 12,
 0.9042297461803984,
 0.8512759824829847,
 0.9136359067720888,
 0.8728146835389444,
 1.0)

In [10]:
watershed = binary_watershed(
    prediction_resized, thres_small=2, rem_seed_thres=1
)
viewer.add_labels(watershed)
eval.evaluate_model_performance(gt_labels_resized, watershed)

2023-03-31 15:37:19,919 - Mapping labels...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3992.79it/s]

2023-03-31 15:37:19,949 - Calculating the number of neurons not found...
2023-03-31 15:37:19,952 - Percent of non-fused neurons found: 54.40%
2023-03-31 15:37:19,953 - Percent of fused neurons found: 34.40%
2023-03-31 15:37:19,953 - Overall percent of neurons found: 88.80%


(68,
 43,
 13,
 10,
 0.8856947654346812,
 0.8747475859219296,
 0.9187750563205743,
 0.862012598981557,
 1.0)

In [11]:
voronoi = voronoi_otsu(prediction_resized, 0.6, outline_sigma=0.7)

from skimage.morphology import remove_small_objects

voronoi = remove_small_objects(voronoi, 2)
viewer.add_labels(voronoi)
voronoi.shape

2023-03-31 15:37:21,076 - build program: kernel 'gaussian_blur_separable_3d' was part of a lengthy source build resulting from a binary cache miss (0.88 s)
2023-03-31 15:37:21,514 - build program: kernel 'copy_3d' was part of a lengthy source build resulting from a binary cache miss (0.42 s)
2023-03-31 15:37:22,021 - build program: kernel 'detect_maxima_3d' was part of a lengthy source build resulting from a binary cache miss (0.50 s)
2023-03-31 15:37:22,642 - build program: kernel 'minimum_z_projection' was part of a lengthy source build resulting from a binary cache miss (0.59 s)
2023-03-31 15:37:23,117 - build program: kernel 'minimum_y_projection' was part of a lengthy source build resulting from a binary cache miss (0.46 s)
2023-03-31 15:37:23,651 - build program: kernel 'minimum_x_projection' was part of a lengthy source build resulting from a binary cache miss (0.52 s)
2023-03-31 15:37:24,188 - build program: kernel 'maximum_z_projection' was part of a lengthy source build resul

(25, 64, 64)

In [12]:
gt_labels_resized.dtype

dtype('int64')

In [13]:
# np.unique(voronoi, return_counts=True)

In [14]:
# np.unique(gt_labels_resized, return_counts=True)

In [15]:
eval.evaluate_model_performance(gt_labels_resized, voronoi)

2023-03-31 15:37:36,854 - Mapping labels...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:00<00:00, 611.96it/s]

2023-03-31 15:37:37,087 - Calculating the number of neurons not found...
2023-03-31 15:37:37,098 - Percent of non-fused neurons found: 87.20%
2023-03-31 15:37:37,104 - Percent of fused neurons found: 1.60%
2023-03-31 15:37:37,114 - Overall percent of neurons found: 88.80%


(109,
 2,
 13,
 8,
 0.8285521200005869,
 0.8809251900364068,
 0.9838709677419355,
 0.782258064516129,
 1.0)

In [16]:
# eval.evaluate_model_performance(gt_labels_resized, voronoi)

2023-03-31 15:40:34,683 - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
